## Loading packages

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Downloading data

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.51 s, sys: 6.63 s, total: 11.1 s
Wall time: 2min 19s


## Extracting ZIP and combining data

In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 14.3 s, sys: 2.5 s, total: 16.8 s
Wall time: 17.1 s


In [6]:
%ls -ltr dailyrainfall/

total 12146232
-rw-r--r--   1 apple  staff  814041183 Mar 28 23:48 data.zip
-rw-r--r--   1 apple  staff   95376895 Mar 28 23:48 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   94960113 Mar 28 23:48 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   82474546 Mar 28 23:48 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  127613760 Mar 28 23:48 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  232118894 Mar 28 23:48 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  330360682 Mar 28 23:48 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  254009247 Mar 28 23:48 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  235661418 Mar 28 23:48 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  294260911 Mar 28 23:48 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  295768615 Mar 28 23:48 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  328852379 Mar 28 2

In [7]:
%%time
### just listing to get an idea how individual file looks like 
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
df = pd.read_csv("dailyrainfall/NorESM2-LM_daily_rainfall_NSW.csv", usecols=use_cols)
df

CPU times: user 650 ms, sys: 71.7 ms, total: 722 ms
Wall time: 724 ms


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.000000,-34.105263,141.25,143.75,7.651974e-07
1,1889-01-02 12:00:00,-36.000000,-34.105263,141.25,143.75,8.955062e-06
2,1889-01-03 12:00:00,-36.000000,-34.105263,141.25,143.75,1.229921e-03
3,1889-01-04 12:00:00,-36.000000,-34.105263,141.25,143.75,1.460834e-03
4,1889-01-05 12:00:00,-36.000000,-34.105263,141.25,143.75,1.145301e-09
...,...,...,...,...,...,...
919795,2014-12-27 12:00:00,-30.315789,-28.421053,151.25,153.75,5.602033e+00
919796,2014-12-28 12:00:00,-30.315789,-28.421053,151.25,153.75,2.049624e+01
919797,2014-12-29 12:00:00,-30.315789,-28.421053,151.25,153.75,7.168385e-02
919798,2014-12-30 12:00:00,-30.315789,-28.421053,151.25,153.75,1.460354e+00


In [10]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
files = glob.glob('dailyrainfall/*.csv')
files.remove('dailyrainfall/observed_daily_rainfall_SYD.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'[^\/]+(?=\.)', file)[0].split("_")[0])
                for file in files)
              )
df.to_csv("dailyrainfall/combined_data.csv")

CPU times: user 5min 37s, sys: 14.6 s, total: 5min 52s
Wall time: 5min 55s


In [11]:
!du -sh dailyrainfall/combined_data.csv

5.6G	dailyrainfall/combined_data.csv


In [12]:
print(df.shape)
df.head()

(62467843, 6)


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


### Time comparison between machines for combining data

Team Member | Operating System | RAM | Processor | Is SSD | Time taken
-- | -- | -- | -- | -- | --
Member 1 | MacOS Monterey  | 16Gb  | i5  | Yes | 5m 55s
Member 2 |macOS Big Sur 11.5.2|   8GB   |  Apple M1 |    yes    |    6min 8s    |
Member 3 | Win 10   |8Gb   | Ryzen 7  | Yes | 11m 17s
Member 4 |   |   |   |   | 

## Loading and reducing memory usage of combined data

### Changing datatype

In [2]:
df = pd.read_csv("dailyrainfall/combined_data.csv", index_col=0)

In [78]:
print(f"Memory usage of complete data: {df.memory_usage().sum() / 1e9:.2f} GB")

Memory usage of complete data: 3.50 GB


In [79]:
df.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [80]:
print(f"Memory usage with float64 of numeric columns: {df[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e9:.2f} GB")


Memory usage with float64 of numeric columns: 3.00 GB


In [81]:
print(f"Memory usage with float32: {df[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e9:.2f} GB")


Memory usage with float32: 1.75 GB


- Changing the datatype from float64 to float32 reduced our data memory usage from 3Gb to 1.75Gb. Almost by half.

In [82]:
%%time
df[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].describe()

CPU times: user 11.7 s, sys: 5.03 s, total: 16.8 s
Wall time: 16.8 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [83]:
df = df[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore')

In [84]:
%%time
df.describe()

CPU times: user 7.61 s, sys: 1.71 s, total: 9.33 s
Wall time: 9.33 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310497e+01,-3.197765e+01,1.469057e+02,1.482150e+02,1.901173e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


- We can observe that after converting our dataframe dtype to float32, our describe function works faster.

### Reading only required columns

In [85]:
# Not reading index
use_cols = ['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
df = pd.read_csv("dailyrainfall/combined_data.csv",
                usecols=use_cols)
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13
1,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13
2,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13
3,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13
4,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13


In [86]:
print(f"Memory usage with float64 of numeric columns without index: {df[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e9:.2f} GB")

Memory usage with float64 of numeric columns without index: 2.50 GB


In [88]:
%%time
# Reading all columns and calling value_counts()
df = pd.read_csv("dailyrainfall/combined_data.csv", index_col=0)
df["model"].value_counts()

CPU times: user 58.2 s, sys: 11.3 s, total: 1min 9s
Wall time: 1min 13s


MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

In [89]:
%%time
# Reading only specific columns and calling value_counts()
use_cols = ['time','model']
df = pd.read_csv("dailyrainfall/combined_data.csv", index_col=0, usecols=use_cols)
df.value_counts()

CPU times: user 43 s, sys: 3.6 s, total: 46.6 s
Wall time: 47.2 s


model           
MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
dtype: int64

### Reading in chunks

In [90]:
%%time
df = pd.read_csv("dailyrainfall/combined_data.csv", index_col=0)
df["model"].value_counts()

CPU times: user 59.7 s, sys: 12.3 s, total: 1min 12s
Wall time: 1min 17s


MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

In [91]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("dailyrainfall/combined_data.csv", chunksize=1_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
#     print(counts.memory_usage())

# print()
# print("Final counts")
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 56.6 s, sys: 6.2 s, total: 1min 2s
Wall time: 1min 2s


- In our case here ,the time taken to calculate the total value counts may not be that significant as reading the whole data into the memory and then performing the operation. 
- But in scenarios where the data cannot fit in the RAM, reading in chunks is the only option.

### Time comparison between machines after reducing memory usage

Team Member | Operating System | RAM | Processor | Is SSD | EDA | Method of optimization |Time before optimization| Time after optimization
-- | -- | -- | -- | -- | -- | -- | -- | -- 
Member 1 | MacOS Monterey  | 16Gb  | i5  | Yes | Describe | Type Conversion|9.12s | 9.33s
Member 1 | MacOS Monterey  | 16Gb  | i5  | Yes | Value_counts | Reading specific columns| 1min 9s | 46.6s
Member 1 | MacOS Monterey  | 16Gb  | i5  | Yes | Value_counts | Chunk processing| 1min 12s | 1min 2s
Member 2 |macOS Big Sur 11.5.2|   8GB |  Apple M1 |    Yes | Describe | Type Conversion|  8.18s | 7.94s
Member 2 |macOS Big Sur 11.5.2|   8GB |  Apple M1 |    Yes | Value_counts | Reading specific columns|  1min 5s | 51.4s
Member 2 |macOS Big Sur 11.5.2|   8GB |  Apple M1 |    Yes | Value_counts | Chunk processing|  1min 5s | 49.1s
Member 3 | Win 10 | 8Gb  | Ryzen 7  | Yes | Describe | Type Conversion|38.1s | 12.5s
Member 3 | Win 10  | 8Gb  | Ryzen 7  | Yes | Value_counts | Reading specific columns| 2min 42s | 1min 36s
Member 3 | Win 10  | 8Gb  | Ryzen 7  | Yes | Value_counts | Chunk processing| 2min 52s | 1min 32s
Member 4 |   |   |   |   |  |  |  |  |

## Perform a simple EDA in R

### Converting Python dataframe to parquet to read in R 

In [3]:
%%time
df.to_parquet("dailyrainfall/combined_data.parquet")

CPU times: user 17.3 s, sys: 2.57 s, total: 19.8 s
Wall time: 20 s


In [4]:
%load_ext rpy2.ipython

Warning message:
In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
  there is no package called ‘devtools’


In [18]:
%%R
library(dplyr)
suppressMessages(library(arrow, warn.conflicts = FALSE))
suppressMessages(library(dplyr, warn.conflicts = FALSE))

### Reading parquet file in R

In [22]:
%%time
%%R
df_r <- open_dataset("dailyrainfall/combined_data.parquet")

CPU times: user 10.1 ms, sys: 14.7 ms, total: 24.8 ms
Wall time: 35.8 ms


- We can observe that reading parquet file is way faster as compared to reading the csv.
- This because of the following reasons :
**Hatef - take it away ;) :D**

### Simple EDA for R 

In [23]:
%%time
%%R
head(df_r)

Table
6 rows x 7 columns
$lat_min <double>
$lat_max <double>
$lon_min <double>
$lon_max <double>
$rain (mm/day) <double>
$model <string>
$time <string>

See $metadata for additional Schema metadata
CPU times: user 7.44 s, sys: 8.29 s, total: 15.7 s
Wall time: 9.64 s


In [35]:
%%time
%%R
df_r |> 
group_by(model) |> 
summarise(count = n()) |> 
arrange(desc(count)) |>
ungroup() |> 
collect() 

# A tibble: 27 × 2
   model           count
   <chr>           <int>
 1 MPI-ESM1-2-HR 5154240
 2 TaiESM1       3541230
 3 NorESM2-MM    3541230
 4 CMCC-ESM2     3541230
 5 CMCC-CM2-SR5  3541230
 6 CMCC-CM2-HR4  3541230
 7 SAM0-UNICON   3541153
 8 GFDL-ESM4     3219300
 9 GFDL-CM4      3219300
10 FGOALS-f3-L   3219300
# … with 17 more rows
CPU times: user 3.82 s, sys: 663 ms, total: 4.48 s
Wall time: 2.16 s
